# Laboratorio 1 - Analisis de ventas Big Data - Ivan Gonzalo Tapia



### Consignas:
Preguntas a responder en el análisis
—————————————————————————————————————————
1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?
2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?

4. Análisis del producto más vendido:

  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?

Pregunta Adicional
—————————————————————————————————————————

   - Además de las preguntas mencionadas anteriormente, te invito a pensar en una pregunta
  adicional que podría ser relevante para analizar los datos de ventas mensuales. Esta
  pregunta debe estar basada en la información que se proporciona en los conjuntos de
  datos. Puede estar relacionada con tendencias, comparaciones, patrones o cualquier otro
  aspecto que despierte tu interés. Trata de ser creativo/a y pensar en cómo podrías explorar
  aún más estos datos!


Pasos del Análisis
—————————————————————————————————————————
1. Carga de datos:
  - Cargar los datos de ventas de cada archivo CSV.
2. Limpieza de los datos:
  - Eliminar datos no numéricos.
  - Eliminar filas incompletas
3. Preparación de los datos:
  - Ajustar los tipos de datos de cada atributo
  - Extraer características importantes para el análisis como meses, horas, ciudades etc.
4. Análisis de datos:
  Ejemplo:
  - Examinar las horas del día en que las ventas son más frecuentes.
  - Determinar el mejor momento para mostrar publicidad.
5. Visualización de datos:
  - Utilizar gráficos y visualizaciones para representar los resultados de los análisis.
  - Crear gráficos de barras, líneas, mapas, tablas, etc, según corresponda.

## Import de librerias y recursos a utilizar

In [184]:
# Librerias necesarias para el desarrollo del proyecto
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


In [185]:
# Directorio propio para importar los datasets necesarios

path = '/content/drive/MyDrive/UPSO ML & BD/Dataset de ventas2'

archivos = os.listdir(path)
archivos

['Dataset_de_ventas_Abril.csv',
 'Dataset_de_ventas_Agosto.csv',
 'Dataset_de_ventas_Diciembre.csv',
 'Dataset_de_ventas_Enero.csv',
 'Dataset_de_ventas_Febrero.csv',
 'Dataset_de_ventas_Julio.csv',
 'Dataset_de_ventas_Junio.csv',
 'Dataset_de_ventas_Marzo.csv',
 'Dataset_de_ventas_Mayo.csv',
 'Dataset_de_ventas_Noviembre.csv',
 'Dataset_de_ventas_Octubre.csv',
 'Dataset_de_ventas_Septiembre.csv']

## Preparacion y limpieza de lo datos

In [186]:
# Ahora voy a crear un diccionario para guardar estos archivos(datasets), para luego hacer un df

dataframes = {}

for archivo in archivos:
  if archivo.endswith('csv'):
    mes = archivo.split('_')[3].split('.')[0] # obtener el nombre del mes solo
    #aca obtenemos la ruta completa de donde se encuentra el csv de cada mes
    path_completo = os.path.join(path, archivo) # obtener ruta de acceso completa
    df = pd.read_csv(path_completo)   #cargar el archivo csv en un df
    #vamos a trabajar sobre una copia del df original
    df_ventas = df.copy()


    # SANITIZAR CADA DF ANTES DE CARGARLO
    df_ventas['Cantidad Pedida'] = pd.to_numeric(df_ventas['Cantidad Pedida'], errors = 'coerce') #errors = coerce transfora lo
    df_ventas['Precio Unitario'] = pd.to_numeric(df_ventas['Precio Unitario'], errors = 'coerce')
    #esta es una mascara que me genera un true si hay un nan, y sino hay nan me pone un false
    df_ventas = df_ventas.loc[~df_ventas['Cantidad Pedida'].isna()]
    df_ventas = df_ventas.loc[~df_ventas['Precio Unitario'].isna()]
    #aca elimino los nan's
    df_ventas = df_ventas.dropna()
    #aca voy a pasar todos los datos al tipo q corresponda y agregar las columnas que sean necesarias
    df_ventas['Cantidad Pedida'] = df_ventas['Cantidad Pedida'].astype(int) #esta funcion me transforma todos los valores a numericos a int
    df_ventas['Precio Unitario'] = df_ventas['Precio Unitario'].astype(float) #esta funcion me transforma todos los valores a numericos a float

    # FINALIZA SANITIZACION

    # CREO COLUMNAS NECESARIAS PARA ANALISIS
    df_ventas['Fecha de Pedido'] = pd.to_datetime(df_ventas['Fecha de Pedido'], format='%m/%d/%y %H:%M')

    # Obtener la hora y minutos como número entero de 4 dígitos (por ejemplo, 1430)
    df_ventas['Hora de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%H%M').astype(int)
    df_ventas['Estado'] = df_ventas['Dirección de Envio'].str.split(', ').str[-1].str.split(' ').str[0]
    df_ventas['Ciudad'] = df_ventas['Dirección de Envio'].str.split(', ').str[1]
    df_ventas['Dia de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%d')
    df_ventas['Mes de Pedido'] = df_ventas['Fecha de Pedido'].dt.strftime('%m')
    #FINALIZA CREACION DE COLUMNAS

    #aca sucede la magia, vamos agregando cada mes al df ventas
    #a cada mes le cargo su dataset correspondiente, por eso uso la variable mes
    dataframes[mes] = df_ventas
    #convierto



dataframe_e = dataframes['Enero'].sort_values(by='Fecha de Pedido', ascending=True)
dataframe_e.head()


,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido,Mes de Pedido
6344,147268,Wired Headphones,1,11.99,2019-01-01 03:07:00,"9 Lake St, New York City, NY 10001",307,NY,New York City,01,01
7154,148041,USB-C Charging Cable,1,11.95,2019-01-01 03:40:00,"760 Church St, San Francisco, CA 94016",340,CA,San Francisco,01,01
8507,149343,Apple Airpods Headphones,1,150.00,2019-01-01 04:56:00,"735 5th St, New York City, NY 10001",456,NY,New York City,01,01
9161,149964,AAA Batteries (4-pack),1,2.99,2019-01-01 05:53:00,"75 Jackson St, Dallas, TX 75001",553,TX,Dallas,01,01
8514,149350,USB-C Charging Cable,2,11.95,2019-01-01 06:03:00,"943 2nd St, Atlanta, GA 30301",603,GA,Atlanta,01,01


In [36]:
dataframe_e.tail()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido
4199,145212,Lightning Charging Cable,1,14.95,2019-02-01 00:36:00,"484 6th St, Dallas, TX 75001",36,TX,Dallas,02-01
6386,147309,Macbook Pro Laptop,1,1700.00,2019-02-01 01:01:00,"594 South St, Dallas, TX 75001",101,TX,Dallas,02-01
5475,146442,USB-C Charging Cable,1,11.95,2019-02-01 01:48:00,"229 West St, Los Angeles, CA 90001",148,CA,Los Angeles,02-01
8137,148987,AA Batteries (4-pack),1,3.84,2019-02-01 02:16:00,"55 Elm St, Atlanta, GA 30301",216,GA,Atlanta,02-01
4364,145372,AAA Batteries (4-pack),1,2.99,2019-02-01 04:02:00,"459 West St, Dallas, TX 75001",402,TX,Dallas,02-01


In [31]:
dataframes['Febrero']

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido
0,150502,iPhone,1,700.00,2019-02-18 01:35:00,"866 Spruce St, Portland, ME 04101",135,ME,Portland,02-18
1,150503,AA Batteries (4-pack),1,3.84,2019-02-13 07:24:00,"18 13th St, San Francisco, CA 94016",724,CA,San Francisco,02-13
2,150504,27in 4K Gaming Monitor,1,389.99,2019-02-18 09:46:00,"52 6th St, New York City, NY 10001",946,NY,New York City,02-18
3,150505,Lightning Charging Cable,1,14.95,2019-02-02 16:47:00,"129 Cherry St, Atlanta, GA 30301",1647,GA,Atlanta,02-02
4,150506,AA Batteries (4-pack),2,3.84,2019-02-28 20:32:00,"548 Lincoln St, Seattle, WA 98101",2032,WA,Seattle,02-28
...,...,...,...,...,...,...,...,...,...,...
12031,162004,Apple Airpods Headphones,1,150.00,2019-02-12 22:02:00,"227 Church St, San Francisco, CA 94016",2202,CA,San Francisco,02-12
12032,162005,AAA Batteries (4-pack),2,2.99,2019-02-04 20:44:00,"417 Jefferson St, Los Angeles, CA 90001",2044,CA,Los Angeles,02-04
12033,162006,USB-C Charging Cable,1,11.95,2019-02-24 06:31:00,"498 8th St, Atlanta, GA 30301",631,GA,Atlanta,02-24
12034,162007,USB-C Charging Cable,1,11.95,2019-02-24 19:09:00,"715 7th St, Dallas, TX 75001",1909,TX,Dallas,02-24


## Analisis y graficos de datos

In [198]:
# Funcion para obtener los datos de dia y hora de los pedidos

def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = df_ventas['Fecha de Pedido'].dt.hour
  datos['Dia'] = df_ventas['Fecha de Pedido'].dt.day
  return datos

In [200]:

# Ordenar los meses
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']

# Crear una lista para almacenar los totales de ventas por mes
totales_ventas = []

# Itera a través de los meses
for mes in meses:
    # Obtén el DataFrame del mes actual
    dataframe_mes = dataframes[mes]

    # Aplica la función para extraer datos y calcular el total de ventas del mes
    dataframe_mes = extraer_datos(dataframe_mes)
    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()

    # Agrega el total de ventas a la lista
    totales_ventas.append(total_ventas_mes)

# Ahora, totales_ventas contendrá los totales de ventas en el mismo orden que la lista de meses
print("Meses:", meses)
print("Totales de Ventas:", totales_ventas)


Meses: ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
Totales de Ventas: [10868, 13461, 17006, 20568, 18657, 15255, 16069, 13442, 13119, 22705, 19808, 28121]


### 1. Comportamiento de las ventas en los distintos meses:

  ● ¿Cómo variaron las ventas a lo largo de los diferentes meses?

  ● ¿Hubo algún mes que se destacó en términos de ventas?

  ● ¿Cuál es el ingreso total generado por mes?

In [86]:
# Creo la figura con plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Bar(
    x=meses,
    y=totales_ventas,
    yaxis="y",
    name="Unidades vendidas",
    marker =dict(color=totales_ventas,colorscale= 'RdYlGn', colorbar= dict(title= "Total de ventas")),
    text = totales_ventas,
    textposition = 'auto'

))

fig.update_layout(
    title="Unidades vendidas por mes",
    xaxis=dict(title="Meses"),
    yaxis=dict(title="Unidades vendidas", side = "left")
)

fig.show()

In [87]:
import plotly.graph_objects as go

sizes = []
colors = []

#Hago un for para recorrer la cantidad total de ventas, y voy a dividir por una constante(1000), para hacer las burbujas de color acorde a la cantidad de unidades vendidas por mes
for total in totales_ventas:
  s = 35
  c = total / 1000
  sizes.append(s)
  colors.append(c)

fig = go.Figure(data=[go.Scatter(
    x=meses,
    y=totales_ventas,
    mode='markers',
    marker=dict(
        color=colors,
        size= sizes,
        showscale=True,
        colorbar= dict(title= "Ventas en miles")
        )
)])

fig.update_layout(

  title="Cantidad de productos vendidos por mes",
  xaxis=dict(title="Meses", tickvals=meses),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [88]:
ingresos_totales = []

for mes in meses:
  total_ventas_mes = (dataframes[mes]['Cantidad Pedida']*dataframes[mes]['Precio Unitario']).sum().round(2)
  ingresos_totales.append(total_ventas_mes)

# Crear el objeto figura
fig = go.Figure()

#Configuro titulos y ejes del grafico
fig.add_trace(go.Bar(
        x = meses,
        y = ingresos_totales,
        yaxis= 'y',
        name = 'Ingreso Total',
        marker = dict(color= ingresos_totales, colorscale='balance', colorbar=dict(title = 'Total de ventas')),
        text = ingresos_totales,
        textposition = 'outside',
)
)

fig.update_layout(
    title = 'Ingresos totales por mes',
    xaxis = dict(title='Meses'),
    yaxis = dict(title='Ingreso total',side='left')
)

#Muestro el grafico
fig.show()

### 2. Optimización de la publicidad y patrón de ventas por hora:

  ● ¿Cuál es el momento ideal del día para presentar la publicidad y aumentar la
  probabilidad de compra?

  ● ¿Cómo cambian los patrones de ventas por hora a lo largo del año?

  ● ¿Hay modificaciones en los patrones de ventas durante las horas de mayor actividad
  en los distintos meses?

In [89]:
# Creo el objeto figura
Horas = list(range(24))
fig = go.Figure()


for mes in meses:

  Ventas_por_hora= dataframes[mes].groupby('Hora')['Cantidad Pedida'].sum()

  # Agrego al objeto figura un grafico de de linea con las horas y las ventas por hora
  fig.add_trace(go.Scatter(

      x=Horas,
      y=Ventas_por_hora,
      mode='lines',
      name=mes,
      line=dict(shape='spline', smoothing=1.3, width=2)))


  # Configuro los ejes y los títulos
fig.update_layout(

    title="Unidades vendidas por hora",
    xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom')),



fig.show()

In [90]:
fig = go.Figure()

horas_analizadas = [12,19,14]

for hora in horas_analizadas:
  total_productos_vendidos = []

  for mes in meses:
    dataframe_mes = dataframes[mes]
    dataframes_mes = extraer_datos(dataframe_mes)

    ventas_hora = dataframe_mes[dataframe_mes['Hora'] == hora]

    total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].sum())

  fig.add_trace(go.Scatter(

      x=meses,
      y=total_productos_vendidos,
      mode='lines',
      name=f'{hora} horas',
      line = dict(width= 2, shape= 'spline')
  ))


# Configuro los ejes y los títulos
fig.update_layout(

  title="Unidades vendidas por hora",
  xaxis=dict(title="Hora", tickvals=Horas),  # Para que en el eje x se muestren todos los valores uso tickvals=Horas
  yaxis=dict(title="Unidades vendidas", side="left"),
  legend=dict(y=0.3, yanchor='bottom')
)


fig.show()

In [ ]:


fig = go.Figure()
fig.add_trace(go.Bar(
    x=meses,
    y=total_productos_vendidos,
    name='Total de unidades vendidas',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=meses,
    y=Ventas_por_hora,
    name='Ventas por hora',
    marker_color='lightsalmon'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

### 2 - Optimizacion de las ventas por hora


In [93]:
ventas_hora = []
# Agrupo las ventas por hora y calculo el total de ventas por hora
for mes in meses:
  ventas_por_hora = dataframes[mes].groupby(dataframes[mes]['Hora de Pedido'])['Cantidad Pedida'].sum().reset_index()
  ventas_hora.append(ventas_por_hora)
# Encuentro la hora con mas ventas
hora_max_ventas = ventas_por_hora.loc[ventas_por_hora['Cantidad Pedida'].idxmax()]['Hora de Pedido']
print(f"La hora ideal para presentar la publicidad es a las {hora_max_ventas} horas.")

La hora ideal para presentar la publicidad es a las 1145 horas.


In [96]:
# Crear un gráfico de barras apiladas para visualizar los patrones de ventas por hora a lo largo del año
import plotly.express as px
ventas_ciudad = []
# Agrupar las ventas por ciudad y calcular el total de ventas por ciudad
for mes in meses:
  ventas_por_ciudad = dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].sum().reset_index()
  ventas_ciudad.append(ventas_por_ciudad)

# Crear un gráfico de barras para visualizar las ventas por ciudad
fig = px.bar(ventas_por_ciudad, x='Ciudad', y='Cantidad Pedida', labels={'Cantidad Pedida': 'Ventas'})
fig.update_layout(title="Ventas por ciudad")
fig.show()

### 3. Distribución de ventas por ubicación:

  ● ¿En qué ciudades se han registrado las mayores ventas?

  ● ¿Cómo se comparan las ventas en diferentes estados o regiones?

  ● ¿Existe variación en las ventas por estado a lo largo de los meses?


In [116]:
# Voy a investigar las ventas por ciudad, obteniendo los datos de la columna ciudad que cree en el procesamiento de datos
for mes in meses:
  ventas_por_ciudad =  dataframes[mes].groupby('Ciudad')['Cantidad Pedida'].sum().reset_index(inplace=False)

ventas_por_ciudad

,Ciudad,Cantidad Pedida
0,Atlanta,2224
1,Austin,1520
2,Boston,3029
3,Dallas,2300
4,Los Angeles,4373
5,New York City,3828
6,Portland,1859
7,San Francisco,6786
8,Seattle,2202


In [117]:
import plotly.graph_objs as go

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Obtener una paleta de colores similar a la usada en Plotly Express
colores_ciudad = px.colors.qualitative.Set1

# Crear un diccionario que mapee cada ciudad a su color correspondiente
color_por_ciudad = {}
for i, ciudad in enumerate(ciudades):
    color = colores_ciudad[i % len(colores_ciudad)]  # Ciclar colores de la paleta
    color_por_ciudad[ciudad] = color

# Agregar el gráfico de barras con colores por ciudad
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Agregar el gráfico de dispersión con los máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color = "red"

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()


In [118]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()






In [119]:
import plotly.express as px

# Calcular las proporciones de ventas por ciudad
ventas_por_ciudad = ventas_por_ciudad.sort_values(by='Cantidad Pedida', ascending=False)
ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
total_ventas = cantidades.sum()
proporciones = cantidades / total_ventas

# Crear un DataFrame para el gráfico
df_pie = pd.DataFrame({'Ciudad': ciudades, 'Ventas': cantidades, 'Proporción': proporciones})

# Crear el gráfico de pastel interactivo
fig = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico
fig.show()




In [112]:
import plotly.graph_objs as go

# Calcular los máximos por ciudad
maximos_por_ciudad = ventas_por_ciudad.groupby('Ciudad')['Cantidad Pedida'].max().reset_index()

# Crear el gráfico
fig = go.Figure()

ciudades = ventas_por_ciudad['Ciudad']
cantidades = ventas_por_ciudad['Cantidad Pedida']
colores_ciudad = px.colors.qualitative.Set1
# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=ciudades,
    y=cantidades,
    mode='lines',
    name="Variacion de unidad vendidas",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por ciudad
for _, row in maximos_por_ciudad.iterrows():
    ciudad_max = row['Ciudad']
    cantidad_max = row['Cantidad Pedida']
    color_max = color_por_ciudad[ciudad_max]  # Obtener el color correspondiente a la ciudad máxima
    fig.add_trace(go.Scatter(
        x=[ciudad_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({ciudad_max})',
        marker=dict(size=10, color=color_max)  # Asignar el color al gráfico de dispersión
    ))


# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)
fig.show()
# Crear el gráfico de pastel interactivo
fig_pie = px.pie(df_pie, names='Ciudad', values='Ventas', color='Ciudad', color_discrete_map=color_por_ciudad,
             labels={'Ciudad': 'Ciudad', 'Ventas': 'Ventas'},
             hover_data=['Proporción'],
             title='Distribución de Ventas por Ciudad')

# Agregar etiquetas de porcentaje
fig_pie.update_traces(textinfo='percent+label')

# Destacar la ciudad con más ventas
fig_pie.update_traces(pull=[0.1] + [0] * (len(ciudades) - 1))

# Configurar el diseño
fig_pie.update_layout(legend_title_text='Ciudades')

# Mostrar el gráfico

fig_pie.show()

fig_bar = go.Figure()
for ciudad in ciudades:
    color = color_por_ciudad[ciudad]  # Obtener el color correspondiente a la ciudad
    fig_bar.add_trace(go.Bar(
        x=[ciudad],
        y=[cantidades[ciudades == ciudad].values[0]],  # Obtener la cantidad para la ciudad específica
        name=ciudad,
        marker_color=color  # Asignar el color al gráfico de barras
    ))

# Configurar ejes y título
fig_bar.update_layout(
    title="Unidades vendidas por ciudad",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig_bar.show()

In [ ]:
# Ahora voy a investigar las ventas totales por estado
for mes in meses:
  ventas_por_estado = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado = ventas_por_estado.reset_index(inplace=False)

ventas_por_estado

,Estado,Cantidad Pedida
0,CA,11159
1,GA,2224
2,MA,3029
3,ME,352
4,NY,3828
5,OR,1507
6,TX,3820
7,WA,2202


In [ ]:
import plotly.graph_objs as go

# Calcular los máximos por estado
maximos_por_estado = ventas_por_estado.groupby('Estado')['Cantidad Pedida'].max().reset_index()
maximos_por_estado = maximos_por_estado.sort_values(by='Cantidad Pedida', ascending=True)
# Crear el gráfico
fig = go.Figure()

estados = ventas_por_estado['Estado']
cantidad = ventas_por_estado['Cantidad Pedida']

# Agregar el gráfico de líneas
fig.add_trace(go.Scatter(
    x=estados,
    y=cantidad,
    mode='lines',
    name="Ciudad",
    line=dict(width=2, shape='spline')
))

# Marcar los puntos máximos por estado
for _, row in maximos_por_estado.iterrows():
    estado_max = row['Estado']
    cantidad_max = row['Cantidad Pedida']
    fig.add_trace(go.Scatter(
        x=[estado_max],
        y=[cantidad_max],
        mode='markers',
        name=f'Máximo ({estado_max})',
        marker=dict(size=10)
    ))

# Configurar ejes y título
fig.update_layout(
    title="Unidades vendidas por estado",
    xaxis=dict(title="Ciudad"),
    yaxis=dict(title="Unidades vendidas"),
)

fig.show()

In [ ]:
import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()

In [ ]:
ventas_por_estado_mes = {}
estados = []


fig = go.Figure()

for estado in dataframes['Enero']['Estado']:
  if estado not in estados:
    estados.append(estado)

for mes in meses:
  ventas_por_estado_mes[mes] = dataframes[mes].groupby('Estado')['Cantidad Pedida'].sum()
  ventas_por_estado_mes[mes] = ventas_por_estado_mes[mes].reset_index(inplace=False)

# Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
  fig.add_trace(go.Bar(
      x=ventas_por_estado_mes[mes]['Estado'],
      y=ventas_por_estado_mes[mes]['Cantidad Pedida'],
      name= mes,
  ));

# Configuro ejes y los títulos
fig.update_layout(
    title="Total de productos vendidos por mes en cada estado",
    xaxis=dict(title="Estados"),
    yaxis=dict(title="Ventas", side="left"),
);

fig.show()

In [ ]:
import pandas as pd

# Supongamos que tienes un diccionario llamado "dataframes" con los DataFrames correspondientes a cada mes
# Puedes cargar los datos desde tu fuente, por ejemplo:
# dataframes = {'Enero': df_enero, 'Febrero': df_febrero, ...}

# Define una función para extraer el estado a partir de la columna "Dirección de Envío"
def obtener_estado(direccion):
    partes = direccion.split(", ")
    if len(partes) > 1:
        return partes[-1]
    else:
        return None

# Inicializa un DataFrame vacío para almacenar los resultados
df_estado_total = pd.DataFrame(columns=['Estado', 'Cantidad Pedida Total'])

# Itera a través de los DataFrames de cada mes en el diccionario
for mes, df_mes in dataframes.items():
    # Aplica la función para obtener el estado y crea una nueva columna "Estado"
    df_mes['Estado'] = df_mes['Dirección de Envio'].apply(obtener_estado)

    # Agrupa por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en este mes
    df_mes_estado_total = df_mes.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

    # Concatena los resultados con el DataFrame total
    df_estado_total = pd.concat([df_estado_total, df_mes_estado_total], ignore_index=True)

# Agrupa el DataFrame total por "Estado" y suma la "Cantidad Pedida" para obtener la cantidad total por estado en todos los meses
df_estado_total = df_estado_total.groupby('Estado')['Cantidad Pedida'].sum().reset_index()

# df_estado_total ahora contiene la cantidad pedida total por cada estado para todos los meses
df_estado_total.head()






,Estado,Cantidad Pedida
0,CA 90001,33289.0
1,CA 94016,50239.0
2,GA 30301,16602.0
3,MA 02215,22528.0
4,ME 04101,2750.0


### 4. Análisis del producto más vendido:
  ● ¿Cuál es el producto más vendido en general y en cada mes?

  ● ¿Qué factores crees que han contribuido al éxito de ese producto en particular?

In [123]:
# Creo un DataFrame general combinando con todos los meses
df_total = pd.concat(dataframes.values(), ignore_index=True)

# Calculo las ventas totales por producto
producto_mas_vendido_general = df_total.groupby('Producto')['Cantidad Pedida'].sum().idxmax()
total_ventas_producto_mas_vendido = df_total.groupby('Producto')['Cantidad Pedida'].sum().max()

print("Producto más vendido en general:", producto_mas_vendido_general)
print("Total de ventas del producto más vendido:", total_ventas_producto_mas_vendido)


Producto más vendido en general: AAA Batteries (4-pack)
Total de ventas del producto más vendido: 31017


In [124]:
# Calcular las ventas totales por producto en el DataFrame combinado
ventas_totales = df_total.groupby('Producto')['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para mostrar las ventas totales por producto
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ventas_totales['Producto'],
    y=ventas_totales['Cantidad Pedida'],
    yaxis="y",
    name="Unidades vendidas",
    marker=dict(color=ventas_totales['Cantidad Pedida'], colorscale='RdYlGn', colorbar=dict(title="Total de ventas")),
    text=ventas_totales['Cantidad Pedida'],
    textposition='auto'
))

fig.update_layout(
    title="Ventas totales de todos los productos en general",
    xaxis=dict(title="Producto"),
    yaxis=dict(title="Unidades vendidas", side="left")
)

fig.show()

In [127]:
# Crear una lista de DataFrames mensuales
dataframes_mensuales = [dataframes[mes] for mes in meses]

# Agregar una columna 'Mes' a cada DataFrame
for i, mes in enumerate(meses):
    dataframes_mensuales[i]['Mes'] = mes

# Concatenar los DataFrames mensuales en un único DataFrame total
df_total = pd.concat(dataframes_mensuales, ignore_index=True)



In [128]:
# Convertir la columna 'Mes' en categórica con el orden deseado
df_total['Mes'] = pd.Categorical(df_total['Mes'], categories=meses, ordered=True)

# Ordenar el DataFrame por la columna 'Mes' categórica
df_total = df_total.sort_values(by='Mes')

In [129]:

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras apiladas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = go.Figure()

for producto in ventas_totales_por_producto_mes['Producto'].unique():
    datos_producto = ventas_totales_por_producto_mes[ventas_totales_por_producto_mes['Producto'] == producto]
    fig_ventas_por_mes.add_trace(go.Bar(
        x=datos_producto['Mes'],
        y=datos_producto['Cantidad Pedida'],
        name=producto
    ))

fig_ventas_por_mes.update_layout(
    title='Ventas totales de todos los productos por mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Cantidad Pedida')
)

fig_ventas_por_mes.show()

In [130]:
import plotly.express as px

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de líneas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = px.line(ventas_totales_por_producto_mes, x='Mes', y='Cantidad Pedida', color='Producto',
                             title='Ventas totales de todos los productos por mes')

fig_ventas_por_mes.update_xaxes(type='category')  # Establecer el eje X como categórico (meses)
fig_ventas_por_mes.show()

In [ ]:
import plotly.express as px

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de áreas para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = px.area(ventas_totales_por_producto_mes, x='Mes', y='Cantidad Pedida', color='Producto',
                             title='Ventas totales de todos los productos por mes')

fig_ventas_por_mes.update_xaxes(type='category')  # Establecer el eje X como categórico (meses)
fig_ventas_por_mes.show()

In [ ]:
import plotly.graph_objs as go

# Calcular las ventas totales por producto y mes
ventas_totales_por_producto_mes = df_total.groupby(['Producto', 'Mes'])['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de dispersión para mostrar las ventas totales por producto y mes
fig_ventas_por_mes = go.Figure()

for producto in ventas_totales_por_producto_mes['Producto'].unique():
    datos_producto = ventas_totales_por_producto_mes[ventas_totales_por_producto_mes['Producto'] == producto]
    fig_ventas_por_mes.add_trace(go.Scatter(
        x=datos_producto['Mes'],
        y=datos_producto['Cantidad Pedida'],
        mode='lines+markers',  # Mostrar líneas y marcadores en el gráfico
        name=producto
    ))

fig_ventas_por_mes.update_layout(
    title='Ventas totales de todos los productos por mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Cantidad Pedida')
)

fig_ventas_por_mes.show()

In [ ]:
# Calcular las ventas totales por mes
ventas_totales_por_mes = df_total.groupby(df_total['Fecha de Pedido'].dt.strftime('%B'))['Cantidad Pedida'].sum().reset_index()

# Crear un gráfico de barras para mostrar las ventas totales por mes
fig_ventas_por_mes = px.bar(ventas_totales_por_mes, x='Fecha de Pedido', y='Cantidad Pedida',
                            title='Ventas totales de todos los productos por mes',
                            labels={'Fecha de Pedido': 'Mes', 'Cantidad Pedida': 'Cantidad Vendida'})

# Configurar el rango del eje Y para mostrar solo hasta 5000
fig_ventas_por_mes.update_yaxes(range=[0, 50000])

fig_ventas_por_mes.show()

In [ ]:
productos_mas_vendidos_por_mes = []

for mes in meses:
    # Obtén el DataFrame del mes actual
    dataframe_mes = dataframes[mes]

    # Agrupa por producto y calcula las ventas totales por producto
    ventas_por_producto = dataframe_mes.groupby('Producto')['Cantidad Pedida'].sum()

    # Encuentra el producto más vendido del mes
    producto_mas_vendido_mes = ventas_por_producto.idxmax()
    total_ventas_producto_mas_vendido_mes = ventas_por_producto.max()

    productos_mas_vendidos_por_mes.append((mes, producto_mas_vendido_mes, total_ventas_producto_mas_vendido_mes))

# Imprime la lista de productos más vendidos por mes
for mes, producto, total_ventas in productos_mas_vendidos_por_mes:
    print(f"Mes: {mes}, Producto más vendido: {producto}, Total de ventas: {total_ventas}")

Mes: Enero, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1596
Mes: Febrero, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1975
Mes: Marzo, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2478
Mes: Abril, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2936
Mes: Mayo, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2931
Mes: Junio, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2160
Mes: Julio, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2366
Mes: Agosto, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 2019
Mes: Septiembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 1927
Mes: Octubre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 3362
Mes: Noviembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 3027
Mes: Diciembre, Producto más vendido: AAA Batteries (4-pack), Total de ventas: 4240


### 5. Tendencia ventas:

  ● ¿Existe alguna tendencia o patrón en las ventas que se repita a lo largo de los días
  en los diferentes meses?

  ● ¿Cómo varían las ventas a lo largo de los diferentes días de la semana?

  ● ¿Se observa alguna diferencia significativa en las ventas entre los días laborables y
  los fines de semana?

In [163]:
dataframes['Enero'].sort_values('Dia de Pedido', ascending = False)

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido,Mes de Pedido,Hora,Dia,Mes
6738,147643,Lightning Charging Cable,3,14.95,2019-01-31 19:39:00,"111 Dogwood St, Los Angeles, CA 90001",1939,CA,Los Angeles,31,01,0.0,17.0,Enero
4653,145655,USB-C Charging Cable,1,11.95,2019-01-31 21:10:00,"312 8th St, San Francisco, CA 94016",2110,CA,San Francisco,31,01,16.0,11.0,Enero
9549,150332,Lightning Charging Cable,2,14.95,2019-01-31 13:21:00,"876 1st St, Atlanta, GA 30301",1321,GA,Atlanta,31,01,11.0,8.0,Enero
8831,149649,ThinkPad Laptop,1,999.99,2019-01-31 09:21:00,"375 Pine St, San Francisco, CA 94016",921,CA,San Francisco,31,01,13.0,29.0,Enero
269,141496,Wired Headphones,1,11.99,2019-01-31 16:21:00,"310 8th St, Los Angeles, CA 90001",1621,CA,Los Angeles,31,01,15.0,17.0,Enero
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,144641,Lightning Charging Cable,2,14.95,2019-01-01 14:06:00,"847 10th St, Boston, MA 02215",1406,MA,Boston,01,01,17.0,5.0,Enero
2504,143610,27in FHD Monitor,1,149.99,2019-01-01 13:58:00,"3 South St, Portland, OR 97035",1358,OR,Portland,01,01,22.0,4.0,Enero
5123,146100,Wired Headphones,1,11.99,2019-01-01 10:02:00,"428 13th St, Austin, TX 73301",1002,TX,Austin,01,01,18.0,17.0,Enero
8095,148948,Lightning Charging Cable,2,14.95,2019-01-01 20:53:00,"926 Forest St, San Francisco, CA 94016",2053,CA,San Francisco,01,01,19.0,2.0,Enero


In [169]:
from plotly.subplots import make_subplots


indice = 1;
dias = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]


fig = make_subplots(rows=1, cols=12, subplot_titles=meses)
for mes in meses:
  for dia in dias:
      dataframe_dia = dataframes[mes].groupby('Dia')['Cantidad Pedida'].sum().max()
      if (dataframe_dia > dataframe_max):
          dataframe_max = dataframe_dia

for mes in meses:
    for dia in dias:
        dataframe_dia = dataframes[mes].groupby('Dia')['Cantidad Pedida'].sum()
        dataframe_dia = dataframe_dia.reset_index(inplace=False)

        # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
        bar_trace1 = go.Bar(
            x=dias,
            y=dataframe_dia['Cantidad Pedida'],
            name=f'{mes}',
        )

        fig.update_xaxes(title_text="Dias", row=1, col=indice)
        fig.update_yaxes(title_text="Cantidad", row=1, col=indice)
        fig.update_yaxes(range=[0, dataframe_max], row=1, col=indice)

    fig.add_trace(bar_trace1, row=1, col=indice)
    indice+=1

# Configuro los ejes y los títulos
fig.update_layout(
    title='Productos por dia por mes',
    width=5000,  # Ancho en píxeles
)

fig.show()

In [179]:

# Definir los días del mes
dias = list(range(1, 32))

# Crear subplots con 3 filas y 4 columnas
fig = make_subplots(rows=3, cols=4, subplot_titles=meses)

for i, mes in enumerate(meses, 1):
    # Obtener el índice del subplot actual
    row = (i - 1) // 4 + 1
    col = (i - 1) % 4 + 1

    # Obtener los datos del DataFrame para el mes actual
    dataframe_mes = dataframes[mes]

    # Calcular el máximo de ventas para este mes
    dataframe_max = dataframe_mes['Cantidad Pedida'].max()

    # Inicializar una lista para almacenar las ventas diarias
    ventas_diarias = []

    # Calcular las ventas diarias para cada día del mes
    for dia in dias:
        ventas_dia = dataframe_mes[dataframe_mes['Dia'] == dia]['Cantidad Pedida'].sum()
        ventas_diarias.append(ventas_dia)

    # Agregar un gráfico de barras para el mes actual
    fig.add_trace(go.Bar(
        x=dias,
        y=ventas_diarias,
        name=mes,
    ), row=row, col=col)

    # Configurar ejes y títulos para cada subplot
    fig.update_xaxes(title_text="Día", row=row, col=col)
    fig.update_yaxes(title_text="Cantidad", row=row, col=col, range=[0, 600])

# Configurar título general
fig.update_layout(
    title='Productos por día por mes',
    width=1500,  # Ancho en píxeles
    height=800,  # Alto en píxeles
)

fig.show()




In [146]:

# Calcular las ventas diarias por día y mes
ventas_diarias_por_mes = df_total.groupby(['Dia de Pedido', df_total['Fecha de Pedido'].dt.month])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras apiladas
fig = px.bar(
    ventas_diarias_por_mes,
    x=ventas_diarias_por_mes.index,
    y=list(ventas_diarias_por_mes.columns),
    barmode='stack',
    title='Ventas Diarias por Día y Mes'
)

# Limitar el eje y a 1000

fig.update_yaxes(range=[0, 1000])

fig.show()



In [64]:
import plotly.graph_objs as go

# Crear una nueva columna 'Mes' a partir de la columna 'Fecha de Pedido'
df_total['Mes'] = df_total['Fecha de Pedido'].dt.month

# Calcular las ventas diarias por mes
ventas_diarias_por_mes = df_total.groupby('Mes')['Cantidad Pedida'].sum()

# Crear un gráfico de barras apiladas
trace = go.Bar(
    x=ventas_diarias_por_mes.index,
    y=ventas_diarias_por_mes.values,
    marker=dict(opacity=0.5),  # Ajustar la opacidad de las barras
)

layout = go.Layout(
    title='Ventas Diarias por Mes',
    xaxis=dict(title='Mes'),
    yaxis=dict(title='Ventas Diarias'),
)

fig = go.Figure(data=[trace], layout=layout)

# Limitar el eje y a 1000
fig.update_yaxes(range=[0, 30000])

fig.show()



In [46]:
import pandas as pd

# Suponiendo que ya has concatenado tus DataFrames en df_total
# Ordenar df_total por la columna 'Fecha de Pedido' en orden ascendente
df_total.sort_values(by='Fecha de Pedido', ascending=True, inplace=True)

# Mostrar las primeras filas del DataFrame ordenado
df_total.columns

Index(['ID de Pedido', 'Producto', 'Cantidad Pedida', 'Precio Unitario',
       'Fecha de Pedido', 'Dirección de Envio', 'Hora de Pedido', 'Estado',
       'Ciudad', 'Dia de Pedido'],
      dtype='object')

In [69]:
df_total5 = pd.concat(dataframes.values(), ignore_index=True)
df_total5.head()

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Hora de Pedido,Estado,Ciudad,Dia de Pedido,Mes de Pedido
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",846,TX,Dallas,19,04
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",2230,MA,Boston,07,04
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,12,04
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",1438,CA,Los Angeles,12,04
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",927,CA,Los Angeles,30,04


In [70]:
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Mes de Pedido', 'Dia de Pedido'])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras apiladas
data = []
for day in ventas_diarias_por_dia.columns:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[day],
        name=f'Día {day}',
        marker=dict(opacity=0.5)  # Ajustar la opacidad de las barras
    )
    data.append(trace)

layout = go.Layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='stack'  # Establecer el modo de las barras
)

fig = go.Figure(data=data, layout=layout)

fig.show()


In [72]:
import plotly.subplots as sp
import plotly.graph_objs as go

# Calcular las ventas diarias a lo largo de todos los meses
ventas_diarias_por_dia = df_total5.groupby(['Mes de Pedido', 'Dia de Pedido'])['Cantidad Pedida'].sum().unstack()

# Crear un gráfico de barras apiladas con barras verticales
fig = sp.make_subplots(rows=1, cols=1)

for day in ventas_diarias_por_dia.columns:
    trace = go.Bar(
        x=ventas_diarias_por_dia.index,
        y=ventas_diarias_por_dia[day],
        name=f'Día {day}',
        opacity=0.5,  # Ajustar la opacidad de las barras
        orientation='v'  # Barras verticales
    )
    fig.add_trace(trace)

fig.update_layout(
    title='Ventas Diarias por Día a lo largo de los Meses',
    xaxis=dict(title='Mes de Pedido'),
    yaxis=dict(title='Ventas Diarias'),
    barmode='stack'  # Establecer el modo de las barras
)

fig.show()


### 6. Impacto de eventos especiales en las ventas:

  ● ¿Se ha observado algún aumento o disminución significativa en las ventas en días
  cercanos a eventos especiales, como días festivos?

  ● ¿Qué eventos específicos han tenido un impacto notable en el comportamiento de
  las ventas y cómo se manifestó ese impacto?